# Project

## Bechdel Test

import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# getting adjacency and edges matrices
adjacency = pd.read_csv('data/adjacency.csv')
#Dropping useless column from adjacency dataframe
adjacency.drop('Unnamed: 0', axis = 1, inplace = True)
adjacency = adjacency.values
np.set_printoptions(suppress = True)
#edges
edges =  pd.read_csv('data/edges.csv')

## Importing from first milestone

In [ ]:
movies = pd.read_csv('data/tmdb_5000_credits.csv')
movies = movies[movies.cast != '[]']

In [ ]:
movies.drop(['title', 'crew'], axis=1, inplace=True)
movies['cast_id'] = movies['cast'].apply(lambda row: list(set(pd.read_json(row)['id'])))

In [ ]:
discount = movies['cast_id'].apply(pd.Series).stack().value_counts()
discount = list(discount[discount > 4].index.astype(int))
discount[:10]

In [ ]:
movies['cast_id'] = movies['cast_id'].apply(lambda x: [y for y in x if y in discount])
movies['edges'] = movies['cast_id'].apply(lambda x: list(itertools.combinations(x, 2)))
edges = list(movies['edges'].apply(pd.Series).stack())
edges[0:5]

edges_df = pd.DataFrame(edges)

In [ ]:
discarded_movies = set()

for idx, movie in movies.iterrows():
    if len(movie['edges']) == 0:
        discarded_movies.add(movie['movie_id'])

print(len(discarded_movies)) 

In [ ]:
movies = movies[~movies['movie_id'].isin(discarded_movies)]
movies.head()

In [ ]:
frames = pd.DataFrame()
new_df = pd.DataFrame()

for idx, film in movies.iterrows():
    cast_df = pd.DataFrame(eval(movies['cast'][idx]))
    cast_df['movies'] = idx
    cast_df = cast_df.drop(['character','order','cast_id', 'credit_id'],axis = 1)  
    
    frames = [new_df, cast_df]
    new_df = pd.concat(frames, join = 'outer', ignore_index=True)

In [ ]:
nodes_df = new_df['movies'].groupby([new_df.gender, new_df.id, new_df.name]).apply(list).reset_index()
nodes_df = nodes_df[nodes_df.id.isin(discount)]
nodes_df

In [ ]:
features = nodes_df.set_index('id').drop('name', axis=1)
features.head()